In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Loading the other modules I have in parent directories
import sys
PYTHONPATH="../"
sys.path.append(PYTHONPATH)
from dataProcessing import getDataLoaders
from models import ThreeCNN, ThreeCNN_Module, rnn_2dCNN
from train import check_accuracy, train, train_ThreeCNN
from plottingFcts import trainingMetrics, sigBkgEff, plotROC
from imageNumber_analysis import num_valAcc 

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
batch_size = 1024
loader_train, loader_val, loader_test = getDataLoaders(batch_size=batch_size, N=90000)

In [4]:
layer0_params_2 = [128, 3, 3, 1, 1]
layer0_reduce_2 = [1, 3, 93, 1, 1] # ---> Downsampling with one filter at the end

layer1_params_2 = [128, 3, 3, 1, 1]
layer1_reduce_2 = [1, 12, 9, 1, 1] # ---> Downsampling with one filter 

layer2_params_2 = [128, 3, 3, 1, 1]
layer2_reduce_2 = [1, 12, 3, 1, 1] # ---> downsampling with one filter 

rnn_params_2 = [18, 18, 1]

layer3_params_2 = [32, 3, 3, 1, 1]

fc_params_2 = [18, 150, 100, 50, 3, 0.5]
print(batch_size*rnn_params_2[1])

18432


In [5]:
rnn_model = rnn_2dCNN(layer0_params_2, layer0_reduce_2, layer1_params_2, layer1_reduce_2, layer2_params_2, layer2_reduce_2, rnn_params_2, layer3_params_2, fc_params_2)

optimizer = optim.Adam(rnn_model.parameters(), lr=1e-5)

hist, bestModel = train(loader_train, loader_val, rnn_model, optimizer, epochs=5, returnBest=True)


Epoch 1/5:
Iteration 0, loss = 1.1539
cuda
Got 8943 / 26999 correct (33.12)



KeyboardInterrupt: 

# Analyzing number of training images vs validation accuracy, using function defined in imageNumber_analysis

In [2]:
layer0_params = [128, 3, 3, 1, 1]
layer0_reduce = [1, 3, 93, 1, 1] # ---> Downsampling with one filter at the end

layer1_params = [128, 3, 3, 1, 1]
layer1_reduce = [1, 12, 9, 1, 1] # ---> Downsampling with one filter 

layer2_params = [128, 3, 3, 1, 1]
layer2_reduce = [1, 12, 3, 1, 1] # ---> downsampling with one filter 

rnn_params = [18, 18, 1]

layer3_params = [32, 3, 3, 1, 1]

fc_params = [18, 150, 100, 50, 3, 0.5]

In [3]:
all_params = [layer0_params, layer0_reduce, layer1_params, layer1_reduce, layer2_params, layer2_reduce, rnn_params, layer3_params, fc_params]

In [4]:
num_images = [10, 100, 1000, 1e4, 2e4, 4e4, 6e4, 7e4, 8e4, 9e4, 1e5]
learn_rate = 1e-5
epochs = 10
batch_size = 64

In [9]:
best_models = num_valAcc(num_images, all_params, learn_rate, epochs, batch_size)

Finished training with 10 images.
Best val accuracy: 0.333333
________________________________________________
Finished training with 100 images.
Best val accuracy: 0.266667
________________________________________________
Finished training with 1000 images.
Best val accuracy: 0.390000
________________________________________________


TypeError: slice indices must be integers or None or have an __index__ method